In [1]:
import os
import sys
current_dir = os.getcwd()
working_dir = os.path.abspath(os.path.join(current_dir, ".."))
code_dir = os.path.join(working_dir, "code")
data_dir = os.path.join(working_dir, "data")
sys.path.append(code_dir)

import datetime
import pickle
from pennylane import numpy as np
import pennylane as qml
from cvqe import gs_cvqe
from utils import ensure_file_dir
from molecules import get_molecule_BeH2

In [ ]:
d_list = [2]

max_iter = 2000
layer = 1

initial_coeffs_strength = 2
cad = True
cad_lr = 0.02
cad_restart_period = 200
threshold_add_bases = 5
random_init = 0.1


for d in d_list:    
    print(f"d: {d}")
    initial_state = {}
    initial_params = None

    lr = d/10
    stepsize = d*0.1

    symbols, geometry, electrons, orbitals, charge = get_molecule_BeH2(d)

    optimizer = qml.GradientDescentOptimizer(stepsize=stepsize)
    params, energies, coeffs, bases, history, grads = gs_cvqe(symbols, geometry, electrons, charge,
                                                            shots=None, max_iter=max_iter, 
                                                            lr = lr,
                                                            cad=cad,
                                                            cad_lr=cad_lr,
                                                            cad_restart_period=cad_restart_period,
                                                            threshold_add_bases = threshold_add_bases,
                                                            layer = layer,
                                                            optimizer=optimizer,
                                                            random_init=random_init,
                                                            initial_coeffs_strength=initial_coeffs_strength,
                                                            initial_state=initial_state,
                                                            initial_params=initial_params,
                                                            dynamic_add_bases=True)

    E = np.min(energies)
    print(f"min energy = {E}")

    file_info = {
        "d": d,
        "layer": layer,
        "stepsize": stepsize,
        "lr": lr,
        "max_iter": max_iter,
        "threshold_add_bases": threshold_add_bases,
        "initial_state": initial_state,
        "initial_params": initial_params,
        "geometry": geometry,
        "symbols": symbols,
        "electrons": electrons,
        "orbitals": orbitals,
        "charge": charge,
        "params": params,
        "energies": energies,
        "coeffs": coeffs,
        "bases": bases,
        "history": history,
        "grads": grads,
        "cad": cad,
        "cad_lr": cad_lr,
        "cad_restart_period": cad_restart_period,
        "initial_coeffs_strength": initial_coeffs_strength,
        "random_init": random_init,
    }

    folder = os.path.join(data_dir,f"BeH2/cvqe_d{d}/")
    datatime_string = datetime.datetime.now().strftime("%Y%m%d%H%M%S-%f") 
    file_path = f"{folder}/info_{d}_iter{max_iter}_{datatime_string}"
    print(file_path)
    ensure_file_dir(file_path)
    with open(file_path, "wb") as f:
        pickle.dump(file_info, f)